In [44]:
import json, re,  os, time, random, time, requests
from bs4 import BeautifulSoup   
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC   
from datetime import datetime, date, timedelta
#import pymongo
import logging  
from lib.jira import jiradriver


In [45]:
jira = jiradriver()
options = {'--headless=new': True, '--window-size=1020,1080': True }   
jira.init(options)  

driver_path: C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\bin\Debug\net48\Chrome\136.0.7103.92\X64\chromedriver.exe


In [ ]:
jira.print_issues()

In [ ]:
jira = jira.get_issues('71').log(time_spent='20m')

In [ ]:
jira.get_issues('BOD').subo() 

In [ ]:
jira.driver.quit()

In [ ]:
jira.driver.get(f'https://dayman.cyber-balance.com/jira/secure/Dashboard.jspa') 

In [ ]:
_= jira.init() 

In [ ]:
posts
df = pd.DataFrame(posts)

In [ ]:
dct = {'driver_path':'xxx'}
if 'driver_path' in dct.keys():
    print('y')
if 'driver_paths' in dct.keys():
    print('y')    

In [ ]:
options = Options() 
options.add_argument("--window-size=1220,980")  
options.add_argument('--log-level=3')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install() ,options=options) 
driver.get(f"https://accounts.craigslist.org/login")  
driver.find_element(By.XPATH, '//input[contains(@id, "Email")]').send_keys('timkopp@gmail.com')
driver.find_element(By.XPATH, '//input[contains(@id, "Password")]').send_keys(ctx.config['clpass']) 
driver.find_element(By.XPATH, '//*[contains(@id, "login")]').click()
time.sleep(2)   
renew = driver.find_elements(By.XPATH,"//form[contains(@action,'7739609697') and contains(@class, 'renew') ]//input[@type='submit']")
if len(renew) > 0:
    #renew[0].click()
    #driver.quit()
    pass

In [ ]:

api_url = "https://dayman.cyber-balance.com/Scheduler/api/atlas/CustomQuery/BOD2201Unremediated/DOE"     
api_url = "https://dayman.cyber-balance.com/Scheduler/api/atlas/CustomQuery/AgencyLevelQuestions/SSA?PKForm=2024-Q1-CIO"  
api_url = "https://dayman.cyber-balance.com/Scheduler/api/atlas/CustomQuery/ErrorLog/?DateFrom=2025-06-24&DateTo=2025-07-07"  

reference_token = "6472A305977D7CE4497CFCF0A510AA6BABCB00DD5CC65E3B2DB32CE8BA085BA1-1"  

headers = {
    "Authorization": f"Bearer {reference_token}"
}
 
try:
    response = requests.get(api_url, headers=headers) 
    response.raise_for_status()  
    json_response = response.json() 
except requests.exceptions.RequestException as e:
    print(f"Error making API request: {e}")
df = pd.DataFrame(json_response)
df


In [ ]:
df.to_html('out/log.html')